In [3]:
import pandas as pd
import torch
import matplotlib.pyplot as plt
from itertools import product
from transformers import BertTokenizer, AdamW
from torch.utils.data import DataLoader, SequentialSampler, RandomSampler
from torch.optim.lr_scheduler import CosineAnnealingLR
from pys.functions import CustomBertModel, create_dataset, train_with_validation, test
from pys.partialData import train_df, val_df, test_df, label_mapping
from pys.params import batch_size, learning_rate, num_epochs

In [4]:
T_max = 50
eta_min = 1e-5

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
train_dataset = create_dataset(train_df, tokenizer, label_mapping)
val_dataset = create_dataset(val_df, tokenizer, label_mapping)
test_dataset = create_dataset(test_df, tokenizer, label_mapping)

model = CustomBertModel(num_labels=len(label_mapping))
optimizer = AdamW(model.parameters(), lr=learning_rate)
scheduler = CosineAnnealingLR(optimizer, T_max=T_max, eta_min=eta_min)
model.to(device)

train_loader = DataLoader(train_dataset, sampler=RandomSampler(train_dataset), batch_size=batch_size)
val_loader = DataLoader(val_dataset, sampler=RandomSampler(val_dataset), batch_size=batch_size)
test_loader = DataLoader(test_dataset, sampler=SequentialSampler(test_dataset), batch_size=batch_size)

f1_train, f1_val, acc_train, acc_val, loss_train, loss_val = train_with_validation(
    model, train_loader, val_loader, optimizer, device, num_epochs, scheduler
)

f1_test, acc_test = test(model, test_loader, device)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\tamar\anaconda3\envs\llm_env\Lib\site-packages\transformers\optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Training Epoch 1/6:   0%|          | 0/29 [00:00<?, ?it/s]

In [ ]:
# Print training results
print("\n--- Training Results ---")
print(f"Training F1 Score (Weighted): {f1_train[-1]:.4f}")
print(f"Training Accuracy: {acc_train[-1]:.2f}%")
print(f"Training Loss: {loss_train[-1]:.4f}")

# Print validation results
print("\n--- Validation Results ---")
print(f"Validation F1 Score (Weighted): {f1_val[-1]:.4f}")
print(f"Validation Accuracy: {acc_val[-1]:.2f}%")
print(f"Validation Loss: {loss_val[-1]:.4f}")

# Print test results
print("\n--- Test Results ---")
print(f"Test F1 Score (Weighted): {f1_test:.4f}")
print(f"Test Accuracy: {acc_test:.2f}%")
